## EDA

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

films.head(3)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00


In [2]:
description.head(3)

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False


### Read all the data into a dictionary of DataFrames

In [3]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()

dfs.keys()

dict_keys(['actor', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store'])

# EDA

- Clean whitespaces

In [4]:
dfs["address"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603 entries, 0 to 602
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   address_id   603 non-null    int64         
 1   address      603 non-null    object        
 2   address2     0 non-null      object        
 3   district     603 non-null    object        
 4   city_id      603 non-null    int64         
 5   postal_code  599 non-null    object        
 6   phone        603 non-null    object        
 7   last_update  603 non-null    datetime64[us]
dtypes: datetime64[us](1), int64(2), object(5)
memory usage: 37.8+ KB


### Strip whitespaces on object-columns and replace whitespace with NaN if cell only contains whitespace

In [6]:
for name, df in dfs.items():
    obj_cols = df.select_dtypes(include="object").columns
    df[obj_cols] = df[obj_cols].apply(lambda col: col.str.strip().replace("", pd.NA))

dfs["address"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603 entries, 0 to 602
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   address_id   603 non-null    int64         
 1   address      603 non-null    object        
 2   address2     0 non-null      object        
 3   district     0 non-null      object        
 4   city_id      603 non-null    int64         
 5   postal_code  599 non-null    object        
 6   phone        0 non-null      object        
 7   last_update  603 non-null    datetime64[us]
dtypes: datetime64[us](1), int64(2), object(5)
memory usage: 37.8+ KB


In [13]:
dfs["address"]["phone"].unique()

array([<NA>], dtype=object)